In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from scipy import spatial

c:\Users\zer0nu11\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [15]:
s = [
 'требования к опыту водительские права категории b d обязанности утренняя развозка персонала две ходки в 09 00 и в 11 00 от м пр т ветеранов до завода росэл в п шушары вечерняя развозка персонала две ходки в 17 00 и в 19 00 от завода росэл до м пр т ветеранов в течении дня выполнение поручений руководства развозка документов материалов небольшого оборудования соблюдение чистоты салона при необходимости осуществление хим чистки салона контроль за прохождением то талонов страховых полюсов комментарий',
 'требования к опыту от 5 лет обязанности развозка документов выполнение личных поручений нег менеджера гостиницы комментарий',
 'требования к опыту о р от 3 лет права водительские кат д обязанности развозка пассажиров на автобусе комментарий',
 'водитель для партнеров',
 'график день ночь 5 2 2 2 6 1 3 1 любой зп в среднем при 5 2 55тр 60 тр в мес',
 'обязанности доставка груза заказчику приемка экспедирование сдача груза грузополучателю оформление погрузочно разгрузочных документов требования опыт работы водителем экспедитором от 5 лет желательно опыт доставки груза в регионы готовность к командировкам до недели действующая мед справка водительское удостоверение категории в с карта водителя для тахографа обязательно желательно международного образца карта естр ответственность условия своевременные выплаты заработной платы бесплатное корпоративное питание исправные автомобили iveco офис в пешей доступности от метро пл восстания пл александра невского официальное трудоустройство график работы пятидневка с 9 до 18 при работе по городу работа по городу и по регионам европейская часть россии',
 'требуются водители на своем авто зарплата 8% от выручки высчитывается от среднего чека количество отвезенных заказов униформу и питание предоставляем',
 'должностные обязанности загрузка отгрузка вверенного оборудования оформление проверка сопроводительных документов работа со складом коммуникация с ответственными лицами требования опыт работы водителем не менее 3 х лет умение читать и проверять накладные отличное знание города опыт работы со складами ответственный подход к работе готовность к использованию личного а м желательно а м не грузовой и не легковой как пример citroen berlingo или jumper водительские права категории b условия работа в компании оказывающей полный комплекс услуг в сфере системной интеграции на предприятиях любых отраслей бизнеса коммерческих и государственных организаций официальное оформление белая з п график работы 5 2 с 9 30 до 18 30 оклад 50 000 руб net амортизация а м от 10000 до 15 000 руб возможность карьерного роста до позиции руководитель отдела логистики',
 'мы предлагаем 120 рублей заказ компенсация гсм 6 рублей км до 25 заказов в день выплаты 1 раз в неделю nbsp ограниченную зону доставки nbsp график работы обсуждается индивидуально трудоутсройство самозанятость вам потребуется доставлять заказы клиентам принимать оплату nbsp требования наличие лмчного автомобиля смартфон на базе android или ios знание города nbsp',
 'оплата согласно сетке',
 'график 6 1 зп от 45 000 необходима возможность работать в дневные вечерние и ночные смены! nbsp',
 'уже более 12 лет является лидером по розничной торговле контрактными запчастями на японские и европейские автомобили магазины и техцентры уже успешно работают в омске екатеринбурге москве и тюмени закуп товаров производится в японии и корее активно растет и развивается и мы ищем сотрудников готовых стать частью нашей высокоэффективной и дружной команды! обязанности прием товара и документов в распределительном центре п большой исток доставка товара клиентам по екатеринбургу и пригороде развоз товара по транспортным компаниям г екатеринбург сдача товара в транспортные компании согласно сопроводительным документам требования наличие грузового автомобиля фургон грузоподъемность 1 5 тонны будет установлен наш gps датчик опыт работы от 1 года на аналогичной должности знание и умение работать с большим объемом товарно сопроводительных документов знание автозапчастей хорошее знание города исполнительность доброжелательность пунктуальность водительские права категории b с условия график работы с 10 00 до 19 00 пятидневная рабочая неделя выходными суббота воскресенье компенсация гсм своевременная выплата заработной платы 2 раза в месяц для сотрудников компании действует дополнительная скидка на товар! опыт вождения права категории b c',
 'немного об nbsp nbsp ndash кто мы nbsp лидеры по розничной торговле контрактными запчастями на японские европейские и корейские автомобили закуп товара производится в японии и корее магазины и техцентры успешно работают в омске москве тюмени екатеринбурге в настоящее время у нас открыта вакансия водителя с личным автомобилем газель обязанности приемка товара автозапчасти и документов на товар от товароведов на складе доставка товара клиентам по городу и пригороду развоз товара по транспортным компаниям верное оформление накладных на сдачу груза в транспортные компании согласно сопроводительных документов требования наличие собственного грузового автомобиля фургон в хорошем состоянии грузоподъемностью 1 5 2 тонны желательно на газу газель или иностранные аналоги с фургоном объем фургона 9 11 кубов на авто будет установлен наш gps датчик опыт работы водителем экспедитором курьером в сфере грузоперевозок транспортные компании или курьерских доставках не менее 1 года водительское удостоверение категории в знание и умение работать с товарно сопроводительными документами хорошее знание города исполнительность дисциплинированность условия график работы 5 2 с 10 00 до 19 00 выходные плавающие заработная плата оклад компенсация гсм своевременная выплата заработной платы 2 раза в месяц для сотрудников компании действует дополнительная скидка на товар район работы поселок большой исток',
 'уважаемый соискатель! благодарим вас за проявленный интерес к нашей компании в обособленном подразделении г санкт петербург север открыта вакансия специалиста на автомобиль lada largus ваши будущие задачи обслуживание и эксплуатация автомобиля оснащенного специализированным оборудованием контроль технического состояния и внешнего вида автомобиля соблюдение правил дорожного движения эксплуатация специализированного оборудования выезд на федеральную трассу к месту проведения мобильного контроля контроль транспортного потока и классификация транспортных средств с помощью специализированного оборудования в точке мобильного контроля обеспечение контроля работы оборудования в том числе устранение возникших нарушений в рамках своей компетенции заполнение отчетных документов путевые листы журнал и т д чтобы стать кандидатом вам нужно высшее или среднее специальное техническое образование опыт управления автомобилем не менее 5 лет действующее водительское удостоверение категории в и медицинская справка по форме 003 в у уверенный пользователь пк мы предлагаем работу в масштабном проекте федерального значения комфортные условия работы оформление в полном соответствии с трудовым кодексом рф достойная заработная плата зарплатные проекты в ведущих банках россии сбербанк альфа банк газпромбанк комфортабельный офис по адресу г санкт петербург график работы сменный 4 2 с 09 00 до 18 00ч 3 дополнительных оплачиваемых дня в год day off забота о здоровье страхование жизни и здоровья от несчастных случаев 24 часа в сутки дополнительная оплата 10 ти дней больничного листа возможность профессионального обучения и карьерного развития поддержка руководителя в адаптационный период внутренние программы on line обучения корпоративная жизнь мероприятия на новый год и день компании детские новогодние подарки возможность участия в отраслевых спортивных соревнованиях',
 'водители на нашем авто с 20 00 до 5 утра',
 'обязанности управление автопогрузчиком требования среднее полное образование опыт работы от 3 х лет на автопогрузчике наличие водительского удостоверения категорий b c и удостоверения тракториста машиниста категории c с отметкой «водитель погрузчика» ответственность дисциплинированность условия день день с 08 00 до 20 30 два дня выходных совокупный доход от 25 678 до 46 733 рублей оклад ежемесячная премия до вычета ндфл оформление по тк рф предоставляется спецодежда корпоративная развозка от до ст м московская авиагородок внимание! в связи с проверкой государственных органов процесс трудоустройства и получения пропуска на контролируемую зону аэропорта может занимать от 6 до 8 недель',
 'обязанности управление служебным автомобилем категория b с системой «паркон» по маршруту патрулирования выявление повреждений имущества и оборудования парковочного комплекса фиксация нарушений пдд поддержание исправного технического состояния автомобиля заправка и мойка взаимодействие с оператором системы «паркон» по обеспечению работоспособности и эксплуатации оборудования требования среднее специальное или высшее образование наличие водительского удостоверения категории b опыт работы от 4 х лет на автомобиле категории b безаварийное вождение отсутствие дтп по вине водителя уравновешенность стрессоустойчивость исполнительность готовность к однообразной работе условия сменный график по согласованию с руководителем совокупный доход от 27 000 до 36 405 рублей оклад ежемесячная премия доплата за ночные смены до вычета ндфл оформление по тк рф корпоративная развозка от до ст м московская авиагородок внимание! в связи с проверкой государственных органов процесс трудоустройства и получения пропуска на контролируемую зону аэропорта может занимать от 6 до 8 недель',
 'водитель на личном авто с графиком 2 2 и более оплата 8% от количества вывезенных заказов',
 'обязанности управление автомобилем категории с d требования среднее средне специальное образование опыт работы от 3 х лет на аналогичной должности водительское удостовение категории с и d навыки по управлению и обслуживанию автомобилей категории с и d ответственность отсутствие вредных привычек наличие удостоверения тракториста машиниста будет преимуществом условия график работы 5 2 в зимний период декабрь март возможен сменный график день ночь оклад ежемесячная премия доплата за ночные смены оформление по тк рф предоставляется спецодежда дмс включая стоматологию после испытательного срока корпоративная развозка nbsp от до ст м московская авиагородок п сиверский гатчина внимание! nbsp в связи с проверкой государственных органов процесс трудоустройства и получения пропуска на контролируемую зону аэропорта может занимать nbsp от 6 до 8 недель срок рассмотрения резюме 14 календарных дней nbsp как правило мы свяжемся с вами раньше но если вы не получили положительного ответа на отклик после 14 дней значит пока мы не готовы сделать вам предложение о работе ваше резюме будет сохранено в нашей базе и в случае появления подходящей вакансии мы обязательно свяжемся с вами по указанному телефону или электронной почте',
 'в круглосуточный ресторан график работы с 18 до 06 с 19 до 07 с 20 до 08 с 21 до 09',
 'авто не легковой а грузовой или грузо пассажирский по стс 2 р в неделю забирать инструмент в транспортной компании и развозить по предприятиям угмк по свердловской области вся зп белая 50 6 амортизация гсм собес онлайн ясиновер булдыгин груз коробки до 10 кг',
 'добрый день [имя кандидата] компания grandee ищет кандидата на вакансию водитель присоединяйтесь к команде профессионалов в одной из ведущих компаний на рынке транспортных услуг! мы предлагаем высокий доход новые комфортабельные автомобили профессиональный рост! такси grandee – новый уровень комфорта! удобно! надежно! быстро! функционально! ваше резюме показалось нам очень интересным высылаем вам информацию по вакансии просим вас ознакомится если наше предложение вам интересно перезвоните пожалуйста в рабочее время в нашем автопарке автомобили ford focus ford galaxy hyundai solaris shkoda rapid все автомобили новые 2015 2016 года выпуска желтые с желтыми номерами застрахованы по каско и осаго наши автопарки м кожуховская варианты работы! работа по договору с ежедневными выплатами без аренды зп от 60000 рублей и более выплаты ежедневные большое количество заказов удобный график суточные планшет то тр комплект летней зимней резины бустер детское кресло зарядные устройства за счет компании должностные обязанности выполнение заказов от диспетчерской обслуживание корпоративных и частных клиентов соблюдение корпоративных стандартов обслуживания пассажиров контроль за техническим состоянием автомобиля чистотой и порядком в салоне требования безаварийный стаж вождения от 3 лет ответственность умение и желание работать с людьми гражданство рф стаж работы водителем не обязателен – проводится обучение планшет и др',
 'в ночные смены с 20 8 19 7 18 6 21 9 22 10 5 2 3 2 3 1 4 2 зп от 50 тыс бесплатная униформа бесплатное питание оплачиваемое обучение',
 'приглашаем в нашу команду nbsp nbsp nbsp энергичных позитивных и целеустремленных! мы вместе учимся и становимся мастерами своего дела будем признательны если в сопроводительном письме вы укажите станцию метро к которой вы ближе всего проживаете что мы предлагаем разные районы города график работы сменный 2 2 3 3 и тд заработная плата часовая оплата за дальность заказа выездные медосмотры обучение внутри компании карьерный рост внутри компании скидки на продукцию компании от 2 х месяцев работы корпоративное питание официальное трудоустройство стабильная выплата з платы 2 раза в месяц наш сотрудник обладает водительскими правами категории в грамотной устной речью личной мед книжкой знанием маршрутов города коммуникабельностью ответственностью вам предстоит доставка продукции компании клиентам на личном автомобиле по стандартам компании тебе точно подойдет наша вакансия если тебя интересует работа подработка работа рядом с домом водитель курьер доставка доставщик официальная зарплата доставка еды высокая оплата труда работа без опыта работа в утреннее дневное вечернее время работа выходного дня работа в своём районе водитель категории в стабильность дружный коллектив и надежность',
 'качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте требования знание дорог г москвы внимательность вежливость аккуратность эмоциональная устойчивость водительские права категории b условия гарантируем стабильность подберем удобный график работы дневные ночные вечерние смены предложим заработную плату % от заказа чаевые обеспечим бесплатными вкусными обедами и красивой униформой',
 'в связи с тем что один водитель уволился боолотбек уулу акылбек и второй водитель замена другому',
 'график 5 2 6 1 с 8 до 20 зп от 50 до 70 возмещение гсм 300р смена',
 'водитель на нашем авто график работы минимальный 2 2 12 так же возможен график 5 2 6 1 оплата 2 раза в месяц зп 25 т р 60 т р',
 'обязанности управление транспортными средствами легковые машины спецтехника автобусы категории b c d наличие nbsp категории aiii будет преимуществом требования образование среднее среднее специальное высшее опыт работы от 3 х лет обязательно nbsp наличие водительских прав категории b c d исполнительность ответственность умение работать в команде стрессоустойчивость условия график 5 2 с декабря по март возможен сменный график работ оклад ежемесячная премия оформление по тк рф дмс включая стоматологию после испытательного срока предоставляется спецодежда корпоративная развозка nbsp от до ст м московская авиагородок п сиверский гатчина внимание! nbsp в связи с проверкой государственных органов процесс трудоустройства и получения пропуска на контролируемую зону аэропорта может занимать nbsp от 6 до 8 недель срок рассмотрения резюме 14 календарных дней nbsp как правило мы свяжемся с вами раньше но если вы не получили положительного ответа на отклик после 14 дней значит пока мы не готовы сделать вам предложение о работе ваше резюме будет сохранено в нашей базе и в случае появления подходящей вакансии мы обязательно свяжемся с вами по указанному телефону или электронной почте',
 '2 человека',
 'график работы 5 2 4 3 на авто компании бензин оплачивает ресторан nbsp зп 45 000 50 000 nbsp гражданство рф строго nbsp стаж вождения от 3 х лет nbsp возраст от 25 лет nbsp свободное владение русским языком nbsp',
 'требуется водитель на своем авто требования возраст от 21 года стаж вождения от двух лет',
 'обязанности управление транспортными средствами категории laquo b raquo laquo c raquo требования среднее общее образование 11 классов опыт работы на автокране от 3 х лет удостоверение машиниста автомобильного крана не ниже 6 го разряда опыт безаварийного вождения на автотранспорте категорий laquo b raquo laquo c raquo наличие 2 ой группы по электробезопа сности будет вашим преимуществом уравновешенность стрессоустойчивость трудолюбие условия пятидневная рабочая неделя с 08 30 до 17 30 оклад ежемесячная премия доплата за вредные условия труда оформление по тк рф дмс включая стоматологию после испытательного срока предоставляется спецодежда корпоративная развозка от до ст м московская авиагородок внимание! в связи с проверкой государственных органов процесс трудоустройства и получения пропуска на контролируемую зону аэропорта может занимать от 6 до 8 недель срок рассмотрения резюме 14 календарных дней nbsp как правило мы свяжемся с вами раньше но если вы не получили положительного ответа на отклик после 14 дней значит пока мы не готовы сделать вам предложение о работе ваше резюме будет сохранено в нашей базе и в случае появления подходящей вакансии мы обязательно свяжемся с вами по указанному телефону или электронной почте',
 'требования образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобное место работы ресторан м сходненская ул свободы предложим разные графики работы предложим высокую заработную плату от 45 000 руб % от заказа чаевые обеспечим бесплатными вкусными обедами и красивой униформой',
 'зп от 70 тр',
 'график 5 2 2 2 стабильный доход nbsp до 100т р бесплатное питание nbsp бесплатная униформа',
 'компания один из ведущих поставщиков для предприятий общепита приглашает водителя экспедитора кат с обязанности управление автомобилем hyundai hd 78 nissan cabstar газель доставка товара продукты питания на автомобиле компании по ресторанам города и в ближайший пригород работа с товарно транспортными документами sbquo учет при приеме сдаче товара на складе и у клиента требования водительские права кат с обязательно! хорошее знание города опыт работы водителем экспедитором по санкт петербургу от 1 года условия база п металлострой ст м рыбацкое график пятидневка выходные суббота и воскресенье оформление согласно тк рф отпуск 28 календарных дней униформа бесплатный медосмотр официальная зарплата заработная плата от 50 000 руб оклад 41 000 руб ежемесячные премии 25 30% от оклада',
 'водитель на авто компании строго рф стаж вождения от 3 лет не моложе 21 года оплата % от вывезенных заказов гсм nbsp',
 'зп 8% от вывезенных заказов график скользящий',
 'опыт реальной работы от 3 лет погрузка и рагрузка автотраспорта с доков и размещение на стеллажах по системе wms до 7 яруса права необходимых категорий со стажем от 5 лет график работы с 07 16 пятидневка адрес склада м обухово пр обуховской обороны 295',
 'обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте требования знание дорог г москвы внимательность вежливость аккуратность эмоциональная устойчивость водительские права категории b условия гарантируем стабильность подберем удобный график работы предложим заработную плату % от заказа чаевые обеспечим бесплатными вкусными обедами и красивой униформой',
 'график работы обсуждается на собеседовании тк должен совпадать с графиком работы сменщика',
 'график работы 5 2 2 2',
 'гражданство рф снг стаж вождения от 3 х лет возраст 25 50 лет мужчина график работы день ночь 2 2 3 3 5 2 нужны как водители в день так и водители в ночь оплата труда при собеседовании в ресторане',
 'график работы 5 2 зп 45 000 50 000 руб на авто компании nbsp бензин оплачивает ресторан nbsp стаж вождения от 3 х лет возраст от 25 лет свободное владение русским языком nbsp',
 'опыт работы от 3 лет работа со стелажами 7 ярус nbsp высотой до 13 м опыт работы с тсд и системой wms',
 'персональный водитель',
 'требуемый опыт работы 3–6 лет полная занятость полный день обязанности доставка продукции на адрес клиенту помощь грузчику требования наличие опыта работы от 3х лет водительский стаж не менее 3х лет права категории в навыки автомеханика ответственность дисциплинированность внимательность пунктуальность условия работа и развитие в одной из самых крупных компаний в своей отрасли оформление по тк рф оплачиваемые отпуска больничные листы корпоративное обучение график работы 5 2 с 9 00 до 18 00 бесплатная парковка разъездной характер работы место работы 10 15 минут от м теплый стан тропарево пос мосрентген',
 'мы ищем nbsp персонального водителя nbsp для одного из руководителей компании сотруднику необходимо будет осуществлять поездки в соответствии с расписанием руководителя обеспечивать безопасное и комфортное вождение автомобиля каждый день проверять чистоту и техническую исправность автомобиля и осуществлять его обслуживание выполнять личные поручения руководителя следить за безопасностью перевозимых пассажиров и сопровождать их до места назначения предупреждать ситуации потенциально несущие опасность для сопровождаемых лиц наши ожидания опыт работы персональным семейным водителем от 3 лет профессиональные навыки вождения отличное знание города санкт петербурга и ленинградской области готовность к графику отклоняющемуся от нормированного исполнительность обязательность пунктуальность неконфликтность соблюдение полной конфиденциальности о любых аспектах трудовой и частной жизни сопровождаемых лиц опрятный внешний вид готовность соблюдать дресс код и деловой этикет наличие собственного автомобиля начало и окончание рабочего дня между центром и севером санкт петербурга водительские права категории b условия работа на автомобиле компании представительский класс ненормированный график работы достойный уровень заработной платы оформление согласно тк рф дмс и оплата спортзала после успешного прохождения испытательного срока мобильная связь с первого дня работы требования к резюме nbsp рассматриваем только резюме с фотографией',
 'обязанности перевозка металлопроката строительных материалов и оборудования по санкт петербургу и ленинградской области редкие рейсы в соседние регионы контроль количества и качества груза фотоотчет с погрузки и выгрузки заполнение сопроводительной документации оперативное выполнение поставленных задач требования опыт работы по категории е знание устройства автомобиля бережное отношение к технике наличие смартфона и умение обращаться с ним высокая работоспособность ответственность пунктуальность условия зарплата сдельная от 70 000 руб собственная ремзона оформление согласно тк рф выплаты заработной платы два раза в месяц без задержек работа на грузовом автомобиле бортовой полуприцеп оплачиваемая стоянка в южной части города',
 'ресторан приглашает на работу водителя курьера на личном авто требования образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на личном автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобное место работы ресторан м преображенская площадь ул преображенская предложим разные графики работы предложим высокую заработную плату от 40 000 руб до 45 000 руб % от заказа чаевые обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования стаж вождения от 3 х лет опыт работы не менее 1 года в подобной должности образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобное место работы ресторан м рязанский проспект предложим разные графики работы предложим высокую заработную плату % от заказа чаевые обеспечим бесплатными вкусными обедами и красивой униформой',
 'требования стаж вождения от 3 х лет образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобный график работы предложим достойную заработную плату обеспечим бесплатными вкусными обедами и красивой униформой',
 'ищем водителя для одного из руководителей должностные обязанности сопровождение руководителя в течение дня на автомобиле бизнес класса выполнение личных поручений руководителя ответственность за содержание автомобиля в чистоте и исправном состоянии требования обязателен опыт работы личным персональным водителем на автомобилях представительского класса отличное знание города из личных качеств важны исполнительность порядочность дисциплинированность бережное отношение к автомобилю презентабельность обязательно соблюдение делового стиля одежды условия старт и финиш петроградская сторона laquo белая raquo заработная плата дмс мобильная связь офис расположен стартовая улица бц аэроплаза рядом пулково 2 график работы нормированный 5 2',
 'в круглосуточный ресторан',
 'примем в коллектив водителя доставки еды на личном авто nbsp график гибкий коллектив дружный питание и униформа бесплатно nbsp заработная плата от 40000 до 80000 руб также возможен карьерный рост nbsp',
 'стаж вождения от 3 х лет образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобный график работы предложим достойную заработную плату обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки на своем авто требования права категории в образование не ниже среднего знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость наличие своего авто обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м славянский бульвар кунцевская график работы дневные смены заработная плата от 40 000 руб % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'приглашает на работу водителя доставки требования стаж вождения от 3 х лет образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность место работы м каширская каширское шоссе 46 подберем удобный график работы предложим достойную заработную плату обеспечим бесплатными вкусными обедами и красивой униформой',
 'обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте требования знание дорог г москвы внимательность вежливость аккуратность эмоциональная устойчивость водительские права категории b условия гарантируем стабильность подберем удобный график работы дневные ночные вечерние смены предложим заработную плату % от заказа чаевые обеспечим бесплатными вкусными обедами и красивой униформой',
 'график 2 2 5 2 6 1 дневные и ночные смены',
 'ресторан приглашает на работу водителя доставки • качественная доставка японской кухни на автомобиле по району прилегающему к ресторану • прием оплаты заказа или расчет по терминалу на месте требования • стаж вождения от 3 х лет • опыт работы не менее 1 года в подобной должности • образование не ниже среднего • знание дорог г москвы • знание правил дорожного движения • знание технических характеристик и устройства автомобиля • внимательность вежливость аккуратность эмоциональная устойчивость водительские права категории b условия • гарантируем стабильность • подберем удобное место работы ресторан щербаковская 50 52 • подберем удобный график работы 2 2 3 1 5 2 по 12ч • предложим высокую заработную плату % от заказа чаевые • обеспечим бесплатными вкусными обедами и красивой униформой полный рабочий день',
 'ресторан приглашает на работу водителя доставки требования права категории в образование не ниже среднего знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м владыкино алтуфьевское шоссе гибкие графики работы заработная плата 40000 45000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки требования права категории в знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость наличие своего авто обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м вднх ул космонавтов гибкие графики работы заработная плата 40000 45000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки на своём автомобиле требования образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобное место работы ресторан м пятницкое ш митинская предложим разные графики работы предложим высокую заработную плату от 45 000 руб % от заказа чаевые',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования стаж вождения от 3 х лет опыт работы не менее 1 года в подобной должности образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобное место работы ресторан м новокосино ул новокосинская предлагаем график работы 2 2 5 2 дневные и ночные смены предложим высокую заработную плату % от заказа чаевые от 40000 руб до 45 000 руб обеспечим бесплатными вкусными обедами и красивой униформой',
 '8% от заказов nbsp график 5 2 6 1 по 12 часов 10 22 12 00 14 02 пт сб наличие медицинской книжки преимущество nbsp',
 'ресторан приглашает на работу водителя доставки обязанности • качественная доставка японской кухни на автомобиле по району прилегающему к ресторану • прием оплаты заказа или расчет по терминалу на месте требования • стаж вождения от 3 х лет • опыт работы не менее 1 года в подобной должности • образование не ниже среднего • знание дорог г москвы • знание правил дорожного движения • знание технических характеристик и устройства автомобиля • внимательность вежливость аккуратность эмоциональная устойчивость водительские права категории b условия • гарантируем стабильность • подберем удобное место работы ресторан м кузьминки • предложим разные графики работы 2 2 4 3 5 2 по 12 часов дневные и ночные смены • предложим высокую заработную плату от 40 000 руб до 45 000 руб % от заказа чаевые • обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки требования права категории в образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость наличие своего авто обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м бибирево ул пришвина разные графики работы заработная плата 40000 45000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования стаж вождения от 3 х лет опыт работы не менее 1 года в подобной должности образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобное место работы ресторан м крылатское осенний бульвар предлагаем график работы 2 2 5 2 дневные и ночные смены предложим высокую заработную плату 60 руб в час % от заказа чаевые от 40000 руб до 45 000 руб обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки на своём автомобиле требования образование не ниже среднего знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность подберем удобное место работы ресторан м крылатское осенний бульвар предложим разные графики работы предложим высокую заработную плату 60 рублей в час от 45 000 руб % от заказа чаевые обеспечим бесплатными вкусными обедами и красивой униформой',
 'график работы 5 2 4 3 nbsp зп 45 000 50 000 nbsp гражданство рф строго',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования права категории в знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле доставки по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м чертаново калужская севастопольская предлагаем разные графики работы с 10 22 с 11 23 с 12 00 с 14 02 заработная плата 40000 45000 115 120 руб с заказа чаевые компенсация бензина 380 руб в день хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования права категории в знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость наличие своего авто обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м бибирево ул пришвина предлагаем разные графики работы дневные и ночные смены заработная плата 40000 45000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'есторан приглашает на работу водителя доставки на авто ресторана требования права категории в знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость наличие своего авто обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м бибирево ул пришвина предлагаем разные графики работы дневные и ночные смены заработная плата 40000 45000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'ночные смены работы с 17 00 до 05 00',
 'водитель доставки на авто компании',
 'водитель на личном авто 2 2 12 часов либо 5 2 12 часов оплата % от то возмещаем расходы на гсм и моб связь предоставляем форменную одежду и вкусно кормим!',
 'строго рф от 21 года стаж по правам от 3 лет график работы 5 2 или 6 1 заработная плата 60 руб час 7% процентов от отвезенных заказов',
 'в ночные дневные смены с 20 8 19 7 18 6 21 9 22 10 9 21 10 22 14 2 16 4 5 2 3 2 3 1 4 2 в день 6% от личной выручки ночью 9% от личной выручки компенсация бензина за 12 часов 460 руб зп от 50 тыс бесплатная униформа бесплатное питание оплачиваемое обучение',
 'требуется водитель на авто компании график по договоренности заработная плата 6% за дневные заказы 9% за ночные гражданство рф стаж вождения от 3 х лет',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования права категории в стаж вождения от 3 х лет знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м жулебино ул привольная график работы дневные смены заработная плата 35000 40000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'гражданство рф стаж вождения более 3 х лет з п 6% день 9% ночь график по договоренности',
 'график работы 5 2 з п от 50000 бензин компенсируется 200 руб',
 'водитель на авто компании зп тариф 60р в час 5% и 8% от личной выручки',
 'требуется водитель на нашем авто на график 5 2 гражданство рф возраст 28 50',
 'требуется водитель nbsp на авто ресторана nbsp график гибкий зар плата от 40000 руб в месяц 8% от заказа гсм и моб связь оплачивается nbsp питание и униформа бесплатно оформление по тк nbsp',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования права категории в знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле доставки по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м чертаново калужская севастопольская ул балаклавский проспект предлагаем разные графики работы с 10 22 с 11 23 с 12 00 с 14 02 заработная плата 40000 45000 115 120 руб с заказа чаевые компенсация бензина 380 руб в день компенсация мобильной связи хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'нужен курьер на нашем авто з п до 40 45',
 'кулаев алексей не хочет работать не устраивает з п орлова наталья маленькая з п орозолиев мактыбек тяжело шигапов наиль маленькая з п',
 'водитель доставки гражданство рф зп от 45000 руб от 21 года стаж вождения от 3 лет на собеседование приходить со среды по воскресенья с 13 00 до 20 00',
 'зп зависит от кол ва отвезённых заказов стоимость 1 заказа 130 руб при доставке по времени вовремя за менее 20 минут 30 руб за заказ ср зп от 35 до 70 тыс руб униформа выдаётся питание бесплатное дружный коллектив есть дневные ночные смены',
 'nbsp нов ше',
 'гибкий график зп от 35000 руб на л а 8% на авто ресторана 6% гсм и телеф связь оплачивается оформление по тк питание и униформа бесплатно дружный коллектив',
 'график 6 1 ночные дневные смены заработная плата от 55 000 компенсация гсм стаж вождения от 3х лет',
 'график 6 1 ночные дневные смены заработная плата от 45 000 компенсация гсм стаж вождения от 3х лет',
 '8% от заказов плюс гсм мы оплачиваем',
 'график работы 2 2',
 '6% от заказов плюс гсм днем 8% плюс гсм ночь',
 '60 р ч 5% 8% от то график 5 2 с ночными сменами стаж во от 3 лет возраст от 25 45 лет',
 'необходим один водитель для ночного времени работы с 22 до 10',
 'необходимы водители только для работы в ночное время с 22 до 10',
 'зп от 50000 рублей',
 'с 1 ноября водитель на авто компании 2 2 12 часов либо 5 2 12 часов оплата % от то возмещаем расходы на гсм и моб связь предоставляем форменную одежду!',
 '– международная компания успешно разрабатывающая умные и функциональные продукты для любых потребностей современного веба cdn хостинг хранение и защита контента от киберугроз потоковые трансляции облачные решения компания основана в 2011 году у нас 4 офиса в люксембурге москве минске перми и собственная уникальная инфраструктура с 37 точками присутствия на 4 континентах в 2013 году наша инфраструктура вошла в книгу рекордов гиннеса установив новый рекорд по одновременному пребыванию игроков на одном mog сервере 190 541 человек в 2014 году спроектированная нами сеть выдержала нагрузку в 1 114 000 одновременно играющих пользователей обязанности \uf02d\tуправление легковым автомобилем lada largus механическая коробка передач \uf02d\tосуществлять доставку оборудования между дата центрами офисом и свх \uf02d\tприемка сборка упаковка выдача отгрузка выгрузка товара согласно заказам и поступлениям \uf02d\tфизическое перемещения товара на складе \uf02d\tразмещение принятого товара по местам хранения \uf02d\tподдерживать складские помещения и все принятые оборудования в надлежащем виде \uf02d\tготовить руководителю отчет по отгруженному оборудованию и принятому оборудованию \uf02d\tвыполнение поручений связанных с доставками для нужд офиса требования \uf02d\tстаж вождения от 3 лет \uf02d\tводительские права категории b \uf02d\tкорректное профессиональное вождение \uf02d\tконтроль за техническим и эстетическим состоянием а м \uf02d\tаккуратность ответственность исполнительность условия \uf02d\tрежим работы с понедельника по пятницу с 10 00 до 19 00 \uf02d\tработу в перспективной стремительно развивающейся международной it компании \uf02d\tофициальное трудоустройство согласно трудовому законодательству российской федерации \uf02d\t белую конкурентоспособную зарплату \uf02d\tпрограмму дмс включает стоматологические услуги',
 'водитель на своем авто 8% от личной выручки средний чек',
 'график работы гибкий 5 2',
 'требуется водитель на нашем авто зп в среднем от 40т при графике 5 2',
 'водитель на личном авто 2 2 12 часов либо 5 2 12 часов оплата % от то предоставляем форменную одежду!',
 'обязанности выполнение работы по установке перемещению подбору приему и выдаче грузов на паллетах с использованием узкопроходного штабелера работа в стеллажной системе узкопроходного типа выполнение погрузочно разгрузочных работ на предприятии вручную или с помощью механизмов предназначенных для погрузочно разгрузочных работ электропогрузчика самоходного электроштабелера гидравлической тележки выполнение работ по внутрискладской обработке материальных ценностей сортировку укладку очистку переноску перевеску участие в рациональном и правильном размещении материалов на складах в соответствии с указаниями руководителя осуществление погрузки материалов для отправки на производство осуществление разгрузки материалов в складские помещения производственного участка и участка упаковки размещая их по видам и сериям осуществлять загрузку готовой продукции с накопительного склада участка упаковки на склад готово продукции или склад ответственного хранения требования опыт работы на складе от 1 года наличие прав для управления знание принципов работы складского оборудования понимание складских процессов',
 'на график 2 2 зарплата на авто ресторана 6% от выручки отвезенных заказов зарплата на личном авто 8% от выручки отвезенных заказов мед книжка обязательно!!!! для граждан снг разрешительные документы',
 'зп тариф ставка 60 р ч % от заказов день 5% ночь 8%',
 'требуется водитель на авто ресторана 6% от выручки гсм и тел связь оплачиваем график гибкий зп от 38000 руб дружный коллектив бесплатное горячее питание карьерный рост',
 'водитель на авто компании 5 2 12 часов оплата % от то возмещаем расходы на гсм и моб связь предоставляем форменную одежду!',
 'водитель на личном авто 5 2 12 часов оплата % от то возмещаем расходы на гсм и моб связь предоставляем форменную одежду!',
 'ресторан приглашает на работу водителя доставки требования права категории в образование не ниже среднего знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на своем автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м верхние лихоборы ул б академическая гибкие графики работы заработная плата 40000 45000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'в связи с тем что один водитель перешел на график выходного дня',
 'водитель на авто компании хочет первестись на личном авто по данной причине открываем вакнсию',
 'ресторан приглашает на работу водителя доставки на авто ресторана требования права категории в стаж вождения от 3 х лет знание дорог г москвы знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность эмоциональная устойчивость обязанности качественная доставка японской кухни на автомобиле по району прилегающему к ресторану прием оплаты заказа или расчет по терминалу на месте условия гарантируем стабильность предлагаем работу в ресторане м жулебино ул привольная график работы дневные смены заработная плата 40000 45000 % от заказа чаевые хороший район много заказов! обеспечим бесплатными вкусными обедами и красивой униформой',
 'гражданство рф снг требования возраст 24 40 лет муж стаж вождения от 3 лет график работы 2 2 3 3 5 2 день ночь оплата труда фикс %',
 'с опытом в доставке',
 'примем в коллектив водителя на личном авто на доставку nbsp условия з п от 40000 руб график гибкий пт и сб ресторан работает до 2х питание и униформа бесплатно nbsp оплачиваемый больничный и отпуск nbsp дружный коллектив nbsp требования грамотная русская речь полный пакет документов стаж вождения от 3х лет nbsp мед книжка nbsp коммуникабельность ответственность nbsp',
 'примем в дружный коллектив водителя для доставки еды на авто ресторана nbsp условия гибкий график nbsp зп от 40000 руб 6% оплачиваемые гсм и моб связь бесплатное питание и униформа карьерный рост требования гражданство рф полный пакет документов паспорт права мед книжка грамотная русская речь начеленность на результат коммуникабельность nbsp',
 'водитель строго рф стаж не менее 3 лет',
 'требуется водитель на автомобиле ресторана требования опыт вождения от 3 х лет возраст от 23 лет опыт работы приветствуется знание правил дорожного движения знание технических характеристик и устройства автомобиля внимательность вежливость аккуратность пунктуальность обязанности доставка японской кухни на автомобиле ресторана по району прилегающему к ресторану прием оплаты заказа расчет по терминалу nbsp условия график работы 5 2 заработная плата % от вывезенных заказов чаевые 2 раза в месяц компенсация гсм сотовой связи обеспечим красивой униформой вкусными обедами ужинами nbsp',
 'необходимы водители только для работы в ночное время с 22 00 до 10 00',
 'увольнение сотрудника семакин илья попал в дтп и решил уволиться nbsp',
 'в связи с запуском проекта пицца не хватает двоих сотрудников',
 'средняя заработная плата 45 тыс руб график 5 2',
 'примем в коллектив водителя на личном автомобиле nbsp график гибкий коллектив дружный зар плата своевременная форма и питание бесплатно условия по з п 8% от выручки гсм и мобильная связь оплачиваются',
 'гражданство рф до 55 лет график работы 5 2',
 'требуется водитель на авто компании график по договоренности зп 6% nbsp от то компенсация гсм мобильной связи предоставление фирменной одежды корпоративное питание',
 'требуется водитель рф',
 'связи с запуском пиццы',
 'требуется nbsp водитель на личном авто график по договоренности зп 8% от то компенсация гсм мобильной связи предоставление фирменной одежды корпоративное питание',
 'стаж работы от 1 года возраст до 40 график 2 2 только в день суббота всегда выходной в месяц примерно 13 смен зп 39 500 т р нетт',
 'график 5 2 6 1 по 12 часов либо с 17 до 24',
 'стаж работы от 1 года возраст до 40',
 'ресторан круглосуточный помимо дневных также ночные смены выплата зп 2 раза в месяц дневные заказы 6% ночные 8% nbsp',
 '% от заказов питание возмещение гсм',
 'требуется водитель на авто компании график смешанный день ночь по 12 часов 6 и 8% от заказов день ночь частичная компенсация гсм телефон на андроиде прошу направлять на собеседование с 12 00 до 16 00',
 'ресторан круглосуточный дневные и ночные смены выплаты зп 2 р в месяц компенсация гсм',
 'водитель на личном авто график работы 6 1 дневные ночные смены зп от 50 т р nbsp 8% от заказа nbsp',
 'в связи с нехваткой nbsp',
 'график работы 5 2 по 12 часов выходные плаваующие nbsp',
 'график 5 2 по 12 часов',
 'график 5 2 6 1 по 12 часов',
 'график 2 2 5 2 6 1 по 12 часов зп 8% от суммы отвезенных заказов 20 р за каждый вовремя доставленный заказ частичаня компенсация гсм форма питание nbsp',
 'водитель на авто компании стаж вождения не менее 3х лет nbsp nbsp график работы по договоренности 5х2 6х1 заработная плата % от личной выручки премия за вовремя доставленные заказы общий доход от 60 000 бесплатное питание форму предоставляем компенсация гсм',
 '08 02 2021 10 02 2021 25 02 2021 11 13 12',
 '08 02 2021 20 02 2021 5 3',
 'водитель nbsp на nbsp нашем nbsp авто с опытом работы nbsp',
 'требуется водитель на авто компании с графиком 5 2 и более зп 6% от количества вывезенных заказов',
 'требуется водитель на авто компании с графиком 5 2 и более зп 6% от кодичества вывезенных заказов',
 'обязательно наличие водительского удостоверения nbsp категории вс достаточно и nbsp только в наличие опыта работы на погрузчике от 1 года возраст от 20 50 лет график работы сменный есть ночные смены',
 'требуется водитель на авто компании с графиком 5 2 дневные и ночные смены зп 6% от количества вывезенных заказов 20 р за каждый своевременно доставленный заказ',
 'требуется водитель на авто компании со стажем от 1 го года вождения рф зп до 55 000 руб nbsp график работы 3 1 5 2 смена 10 12 часов nbsp',
 'обязанности работа на автомобилях компании газель цельнометаллическая nbsp lada largus с кондиционером доставка груза клиентам прием сдача товара в транспортные компании осуществлять погрузо разгрузочные работы доставка тмц по москве и области выполнять поручения руководителя поддерживать автомобиль в исправном состоянии содержать автомобиль в чистоте проводить техническое обслуживание автомобиля требования опыт работы от 3 х лет на аналогичной должности знание нормативной документации знание автозапчастей хорошее знание города исполнительность доброжелательность пунктуальность водительские права категории b условия график работы с 9 00 до 19 00 пятидневная рабочая неделя с двумя плавающими выходными заработная плата оклад премия своевременная выплата заработной платы 2 раза в месяц для сотрудников компании действует дополнительная скидка на товар',
 'в большой дружный коллектив т требует водитель на авто компании наши условия зп 6% от средного чека заказа 20 рублей к заказу если его доставили вовремя nbsp бензин за счет компании ремонт автомобиля nbsp мойка автомобиля униформа для сотрудников двухразовое питание обед ужин своевременная зарплата и аванс от сотрудника на данную должность требуется наличие водительских прав полный пакет документов стаж вождения от 3 х лет и более',
 'возраст 35 43 nbsp стаж вождения 10 лет опыт работы персональным водителем от 5 лет желательно из такси nbsp',
 'требуется водитель на авто компании зарплата 8% от всех доставленных заказов доплата за вовремя доставленные заказы возмещение гсм за смену nbsp питание форма',
 '200р заказ в среднем 60тр ночные и дневные смены',
 '5 2 по 12 часов пр данном графике зп от 50т р',
 'требуется водитель доставки на авто компании 175р за заказ оплата гсм nbsp гражданство рф без вредных привычек nbsp',
 'требуется водитель на авто компании требования граждане рф стаж вождения от двух лет не моложе 21 года',
 'компания ndash российский системный интегратор и поставщик ит решений в связи с расширением бизнеса ищет водителей экспедиторов приглашаем на работу водителя экспедитора со своим автомобилем требуемый опыт работы от 1 года полная занятость обязанности своевременная доставка груза по москве и московской области до 10 доставок в день аккуратное вождение и соблюдение пдд ответственность за состояние груза оказание высокого уровня сервиса нашим клиентам взаимодействие со смежными подразделениями компании требования опыт работы на аналогичной должности от 1 года водительское удостоверение категории обязательно права российские наличие доступа за ттк условия уровень оплаты nbsp обсуждается местоположение комфортный офис недалеко от nbsp м беговая nbsp шаговая доступность рабочий день nbsp с 09 00 до 18 00 гарантии nbsp оформление по тк рф и полное его соблюдение компенсации nbsp оплата мобильной связи оплата переработок по тк рф дополнительная оплата в случае разгрузки машины компенсация бензина платных парковок и платных въездов оплачиваемый отпуск 28 календарных дней ключевые навыки вождение и соблюдение пдд ответственность и исполнительность пунктуальность',
 'график 5 2 6 1 с 8 до 20 зп от 40 до 60',
 'дневные и ночные смены график работы 2 2 3 2 3 3 4 2 5 2 6 1',
 'график 5 2 6 1 по 12 часов сдельная оплата за заказ nbsp',
 'график 5 2 2 2 есть ночные смены бесплатное питание nbsp стабильный заработок',
 '200р заказ 530 оплата бензина зп от 80 000 с графиком 5 2 по 12 часов',
 'график работы 5 2 по 12 часов стоимость заказа 175 руб nbsp',
 'график работы 5 2 6 1 доход от 80т р в месяц компенсация гсм 250р смена предоставляется униформа и питание',
 'гражданство рф график 5 2 бесплатное питание nbsp стабильный доход бесплатная униформа',
 'в дружную компнду требуется воитель на лично авто мы предлагаем двухразовое питание своевременную выплату зарплаты оплата 8% от заказа оформление медицинской книжки предоставление униформы nbsp',
 'график работы 2 2 3 1 6 1 5 2 дневные и ночные смены nbsp',
 'график работы 2 2 5 2 6 1 по 12 часов зп 190 200 р за 1 заказ компенсация гсм компенсация мобильной связи питание форма инструменты для работы беплатно уровень зп высокий!',
 'график 6 1 5 2 ресторан суточный',
 'график по 12 часов зп от 75 тр',
 'график 5 2 2 2 6 1 190р заказ в среднем в месяц доход от 80тыс при графике 5 2 по 12ч питание 2 раза в день униформа возмещение гсм 250р смена от 12часов',
 'график работы 5 2 6 1 рассмотрим по совмещению nbsp обед ужин nbsp отвозить заказы гостям график работы по 12 часов',
 'водитель категории е на тягач hyundai hd 500 компания laquo raquo предоставляет услуги аренды спецтехники мы работаем на рынке nbsp более 5 лет в связи с увеличением количества заказов и расширением деятельности мы ищем водителя категории laquo е raquo для работы на тягаче hyundai hd 500 nbsp что вы получите индивидуальный график работы в соответствии nbsp с режимом работы заказчика работа по г москве и московской области без командировок заработная плата сдельная от 80000 до 110000 рублей на руки иногородним предоставляется жилье nbsp ваши обязанности nbsp управление грузовым автомобилем с полуприцепом заполнение путевых листов и товарно транспортных накладных готовность к ненормированному рабочему дню nbsp требования nbsp права категории с е опыт работы не менее 1 года ответственность дисциплинированность',
 'обязанности перемещение автомобилей на территории дилерского центра ведение отчетности требования водительские права категории в безаварийный стаж вождения от 5 nbsp лет внимательность аккуратность открытость честность желание качественно работать и развиваться условия престижную работу в крупнейшем автохолдинге россии официальное трудоустройство по тк рф официальную laquo белую raquo заработную плату уникальную возможность построить карьеру а также регулярно проходить обучение внутри компании график работы сменный 2 2 3 место работы котельники новорязанское шоссе дом 1 г',
 'трудоустройство по тк рф вся зар плата белая з п 65 000 руб до вычета налогов график работы 5 2 с 09 00 18 00 место работы м фили береговой проезд д 2 стр 2 обязанности управлять погрузчиками и всеми специальными грузозахватными механизмами и приспособлениями при погрузке выгрузке перемещении и укладке грузов производить техническое обслуживание погрузчика и текущий ремонт всех его механизмов определять неисправности в работе погрузчика устанавливать и заменять съемное оборудование участвовать в проведении планово предупредительного ремонта погрузчика грузозахватных механизмов и дополнительного оборудования требования знание технической части погрузчика опыт работы на аналогичном виде техники знание пдд в у с соответствующими допусками',
 'требуется водители в круглосуточный ресторан на график 5 2 только в ночную смену',
 'график 5 2 или 6 1 дневные и ночные смены 190 nbsp руб за заказ оплата гсм 500 руб за смену совокупный доход за месяц 70 80т р',
 'график 5 2 по 12 часов ночная смена бесплатное питание бесплатная униформа стабильный доход',
 'доставка заказа от ресторана гостю оплата 175 р заказ смена по 12 часов дневные и ночны смены график индивидуальный предоставляем скутер брендированую форму обязательно с правами с отвтествующей катигорией nbsp',
 'мы международная логистическая компания которая представлена в 25 филиалах по россии nbsp более чем в 16 странах мира мы ищем сотрудников которые будут представлять лицо нашей компании и являться основным связующим звеном с нашими клиентами сдэк приглашает на работу водителя погрузчика на склад у тебя будет очень важная роль mdash ты будешь отвечать за то чтобы наши любимые клиенты получили свои посылки вовремя а для этого важно управление погрузчиком и выполнение погрузочно разгрузочных мероприятий участие в укладке грузов и складской тары их креплении и размещении уход за погрузчиком и мониторинг качества ремонта погрузчика мы предлагаем работу в известной международной компании место работы nbsp ст м варшавская график работы 4 4 nbsp 2 дня в день 7 00 до 19 00 2 дня в ночь с 19 00 до 7 00 4 выходных дня официальное оформление по трудовой книжке возможность карьерного роста оплата за отработанные дневные и ночные смены в соответствии с нормой часов ndash 54 960р 1 месяц со 2 го месяца бонус 40% 69 000 70 000р крутые корпоративы от нашей дружной компании чтобы работать у нас тебе нужно совсем немного обязательное наличие прав на управление погрузчиком внимательность и оперативность любить и уметь впитывать в себя новую информацию как губка отправляй нам своё резюме или звони по контактам указанным в вакансии ! сдэк компания про людей !',
 'мы занимаемся поставкой строительных и отделочных материалов и являемся одним из лидеров российского рынка работая с нашей компанией вы будете получать только качественные материалы зарекомендовавшие себя за годы эксплуатации вы освобождаетесь от общения с перекупщиками и завышенных цен так как мы работаем только с заводами производителями накопив огромный опыт работы обеспечиваем любой строительный объект всем необходимым клиентам приобретающим строительные материалы в нашей компании обеспечиваем качество и надёжность в распоряжении имеется автопарк грузового транспорта nbsp обязанности nbsp загрузка строительного материала в городах егорьевск воскресенск брянск москва выгрузка на объектах москвы и мо следить за техническим состоянием управляемого транспорта и своевременно сообщать в случае сбоев умение работать с документами при загрузке и выгрузки своевременно сдавать их в бухгалтерию требования nbsp права категории се прохождение медосмотра от 2 3 лет опыта вождения на данной категории условия nbsp график 6 1 время обсуждается с руководителем гсм оплачивается машина камаз 20 тн тягач и прицеп бортовой оформление по тк рф nbsp',
 'примем в штат водителя на личном авто nbsp график гибкий nbsp питание и униформа бесплатно nbsp зп от 65000 руб nbsp гсм и интернет оплачиваются nbsp',
 'форма питание оплата 175 190 заказ график индивидуальный режим работы доставки 10 24',
 'примем в наш дружный коллектив водителя на личном авто зп 2 раза в месяц питание и униформа бесплатное',
 'оплата сдельная заказ 200 р график 5 2 6 1 режим работы ресторана 10 00 24 00 права категории в стаж вождения от 1 года грамотная русская речь коммуникабельность',
 'график работы 5 2 6 1 ставка за заказ 190р среднее кол во заказов в месяц 350шт совокупный доход 60 70тр предоставляется униформа питание',
 'вам предстоит выполнение поручений выезд на корпоративном автомобиле график 3 3 с проживанием 4 часа на сон в сутки оплата 6 тысяч выполнение поручений руководителя sbquo разъездного характера выезды на другие объекты компании в рамках поручения руководителя своевременное отслеживание технического состояния автомобиля содержание автомобиля в чистоте и исправном состоянии 2 обеспечение порядка на объекте бц переодический обход территории с целью поддержания установленных норм корпоративных правил на территории бц контроль пропусков физ лиц и автотранспорта сопровождение сотрудников компании на деловые встречи и переговоры в случае необходимости мы ожидаем уверенный пользователь пк ms word ms excel outlook наличие улчо необязательно оказываем помощь в получении наличие прав категории в обязательно готовность финального кандидата к прохождению полиграфа! условия работа в стабильной компании более 30 лет на рынке официальное оформление с первого рабочего дня льготы и компенсации в соответствии с тк рф заработная плата 6000 рублей в сутки премии ежегодная индексация надбавки график работы суточный сменный 3 3 устанавливается индивидуально помощь в получении улчо удостоверение личности частного охранника скидка 30% 50% в фитнес клуб с бассейном',
 'гибкий график стабильный доход бесплатное питание бесплатная униформа полный рабочий день',
 'требуется водитель на своем авто рассмотрим кандидатов в возрасте от 21 года со стажем вождения от двух лет на собеседовании иметь полный комплект документов',
 'компания nbsp nbsp в поисках водителя теха работаем на корпоративном автомобиле citroen berlingo по обслуживанию физических лиц и автомобилей каршеринга база находится на станции метро волгоградский проспект обязанности большая часть заявок предполагает собой выездные услуги по замене колес в сборе далее исходя из ваших навыков возможны заявки следующего типа проведение мероприятий по запуску двигателя транспортного средства помощь в дозаправке транспортного средства работы по замене колеса транспортного средства снятие тс с паркинга требования знание устройства автомобиля навыки замены колес стаж вождения от 3х лет условия работа в надежной и стремительно развивающейся компании с возможностью профессионального и карьерного роста внутри оклад за каждую заявку заявок очень много доход не ограничен официальное оформление согласно тк рф удобный график 6 смен отработали 6 смен отдыхаете можно брать подработки nbsp с 21 00 09 00 в ночь корпоративные скидки на услуги компании фитнес и туризм покупка и обслуживание авто внимание! работа предполагает физический труд в любую погоду просьба не откликаться людей не готовых к таким условиям',
 'обязанности эвакуация легковых автомобилей компаний и частных лиц оформление сопроводительной документации требования опыт работы по категории с не менее 1 года ответственность желание работать и зарабатывать знание пдд опыт работы на эвакуаторе не обязателен внутри компании возможно обучение! условия официальное оформление согласно тк рф отпускные больничные график работы 2 2 с 9 00 до 21 00 день ночь работа на новом корпоративном эвакуаторе газон next со сдвижной платформой возможность бесплатного проживания на территории компании возможность оставлять машину laquo у дома raquo обсуждается на собеседовании стабильные и своевременные выплаты заработной платы справедливое и адекватное руководство корпоративные скидки на услуги компании туризм покупка и обслуживание авто услуги недвижимости',
 'развоз заказов питание униформа зп без задержек график работы индивидуальный смена по 12 часов доствка nbsp работает с 10 00 до 23 30',
 'график 5 2 6 1 nbsp',
 'требуются водители на личном авто стаж вождения от двух лет на собеседовании с собой иметь полный комплект документов',
 'график работы 5 2 6 1 смены по 12часов оплата за заказ 190р частичная компенсация гсм совокупный доход от 70тр',
 'ресторан круглосуточный график работы 5 2 6 1 время работы с 18 00 до 6 00 с 19 00 до 07 00 с 20 00 до 08 00 заработная плата от 60000 плюс 400 гсм nbsp',
 'ресторан круглосуточный график работы 5 2 6 1 с 18 00 до 6 00 с 19 00 до 07 00 с 20 00 до 08 00 nbsp',
 'график работы 5 2 6 1 с 08 00 до 20 00 с 07 00 до 19 00 nbsp',
 'требуются водители на личном авто график работы 5 2 6 1 190 руб за заказ возмещение гсм оплата мобильной связи курьерского приложения бесплатное питание и униформа',
 'график работы 6 1 либо 5 2 дневные и ночные смены по 12 ч',
 'требуется на водитель nbsp на автомобиле компании график работы сменный по 12 часов nbsp заработная плата день 6% от ср чека ночь с 24ч 9% от ср чека',
 'требуется водитель на своем авто возраст от 21 года стаж вождения не менее 2 лет на собеседовании при себе иметь полный nbsp пакет документов',
 'водитель на авто компании стаж вождения не менее от 3 лет возраст от 21г',
 'водитель возраст от 21г стаж от 3 лет',
 'водитель погрузчика штабелера nbsp nbsp обязанности прием отпуск товара со склада комплектование товара размещение товара на стеллажах участие в инвентаризации nbsp требования опыт работы на спецтехнике от 1 года наличие прав тракториста машиниста кат в умение работать с терминалом сбора данных nbsp условия оформление по тк рф здоровье дмс со стоматологией страхование от несчастных случаев полис выезжающих за рубеж специальные программы страхования для родственников питание компенсация обедов график работы пн пт с 9 00 до 17 30 расположение каширское шоссе апаринки ближайшее метро домодедовская',
 'уважаемый соискатель! благодарим вас за проявленный интерес к нашей компании в обособленном подразделении г сыктывкар открыта вакансия специалиста смк на автомобиль уаз патриот на период отпусков основных сотрудников ваши будущие задачи обслуживание и эксплуатация автомобиля оснащенного специализированным оборудованием контроль технического состояния и внешнего вида автомобиля соблюдение правил дорожного движения эксплуатация специализированного оборудования выезд на федеральную трассу к месту проведения мобильного контроля nbsp контроль транспортного потока и классификация транспортных средств с помощью nbsp специализированного оборудования в точке мобильного контроля обеспечение контроля работы оборудования в том числе устранение возникших нарушений в рамках своей компетенции заполнение отчетных документов путевые листы журнал и т д чтобы стать кандидатом вам нужно высшее или среднее специальное техническое образование опыт управления автомобилем не менее 5 лет действующее водительское удостоверение категории в и медицинская справка по форме 003 в у уверенный пользователь пк мы предлагаем работу в масштабном проекте федерального значения комфортные условия работы оформление в полном соответствии с трудовым кодексом рф по срочному трудовому договору на период с 07 по 28 июля nbsp достойная заработная плата зарплатные проекты в ведущих банках россии сбербанк альфа банк газпромбанк комфортабельный офис по адресу г сыктывкар график работы сменный с 08 00 до 17 00ч 3 дополнительных оплачиваемых дня в год day off забота о здоровье страхование жизни и здоровья от несчастных случаев 24 часа в сутки дополнительная оплата 10 ти дней больничного листа возможность профессионального обучения и карьерного развития поддержка руководителя в адаптационный период внутренние программы on line обучения корпоративная жизнь мероприятия на новый год и день компании детские новогодние подарки возможность участия в отраслевых спортивных соревнованиях']
s1 = s[0]
s = s[1:]
S = []
S1 = embed_bert_cls(s1, model, tokenizer)
for name in s:
    S.append(1-spatial.distance.cosine(S1,embed_bert_cls(name, model, tokenizer)))
res = pd.DataFrame(data={'name':s,'dist':S})
res = res.sort_values(by=['dist'],ascending=False).reset_index(drop=True)

In [16]:
s1

'требования к опыту водительские права категории b d обязанности утренняя развозка персонала две ходки в 09 00 и в 11 00 от м пр т ветеранов до завода росэл в п шушары вечерняя развозка персонала две ходки в 17 00 и в 19 00 от завода росэл до м пр т ветеранов в течении дня выполнение поручений руководства развозка документов материалов небольшого оборудования соблюдение чистоты салона при необходимости осуществление хим чистки салона контроль за прохождением то талонов страховых полюсов комментарий'

In [22]:
res.loc[160]['name']

'ресторан круглосуточный график работы 5 2 6 1 с 18 00 до 6 00 с 19 00 до 07 00 с 20 00 до 08 00 nbsp'

In [18]:
res

,name,dist
0,обязанности управление автопогрузчиком требова...,0.922842
1,требуется водитель на автомобиле ресторана тре...,0.916788
2,обязанности управление служебным автомобилем к...,0.906334
3,обязанности доставка груза заказчику приемка э...,0.904785
4,ресторан приглашает на работу водителя доставк...,0.901963
...,...,...
209,персональный водитель,0.306974
210,водитель для партнеров,0.306224
211,в круглосуточный ресторан,0.301646
212,nbsp нов ше,0.249049
